In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yashkumar2412","key":"110ff56020b3663dabc0301d71021a49"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!ls /root/

In [5]:
!kaggle competitions download -c ieee-fraud-detection

 90% 106M/118M [00:00<00:00, 248MB/s] 
100% 118M/118M [00:00<00:00, 217MB/s]


In [6]:
!unzip ieee-fraud-detection.zip

Archive:  ieee-fraud-detection.zip
  inflating: sample_submission.csv   
  inflating: test_identity.csv       
  inflating: test_transaction.csv    
  inflating: train_identity.csv      
  inflating: train_transaction.csv   


In [7]:
!python -m pip install -q --upgrade pip
!python -m pip install -q autogluon

In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:
!ls /content/

AutoGluonModels		  sample_data		 test_transaction.csv
ieee-fraud-detection.zip  sample_submission.csv  train_identity.csv
kaggle.json		  test_identity.csv	 train_transaction.csv


In [2]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor

directory = '/content/' # directory where you have downloaded the data CSV files from the competition
label = 'isFraud'  # name of target variable to predict in this competition
eval_metric = 'roc_auc'  # Optional: specify that competition evaluation metric is AUC
save_path = directory + 'AutoGluonModels/'  # where to store trained models

train_identity = pd.read_csv(directory+'train_identity.csv')
train_transaction = pd.read_csv(directory+'train_transaction.csv')

In [3]:
train_data = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
train_data.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [4]:
train_data.shape

(590540, 434)

In [5]:
train_data = train_data[:10000] # subset of training dataset to avoid 'Out of memory' error
train_data.shape

(10000, 434)

In [6]:
predictor = TabularPredictor(label=label, eval_metric=eval_metric, path=save_path, verbosity=3).fit(
    train_data, presets='good_quality', time_limit=120
)

results = predictor.fit_summary()

Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
GPU Count:          1
Memory Avail:       5.72 GB / 12.67 GB (45.1%)
Disk Space Avail:   65.20 GB / 112.64 GB (57.9%)
Presets specified: ['good_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': True,
 'num_bag_sets': 1,
 'refit_full': True,
 'save_bag_folds': False,
 'set_best_to_refit_full': True}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': True,
 'calibrate': 'auto',
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 0.25,
             'enable_ray_logging': True,
             'holdout_data': None,
             'holdout_frac': 0.111111

(_ray_fit pid=13730) [50]	valid_set's binary_logloss: 0.0740525


(_ray_fit pid=13823) /usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning:  [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(_ray_fit pid=13823) Dask dataframe query planning is disabled because dask-expr is not installed. [repeated 2x across cluster]
(_ray_fit pid=13823)  [repeated 4x across cluster]
(_ray_fit pid=13823) You can install it with `pip install dask[dataframe]` or `conda install dask`. [repeated 2x across cluster]
(_ray_fit pid=13823) This will raise in a future version. [repeated 2x across cluster]
(_ray_fit pid=13823)   warnings.warn(msg, FutureWarning) [repeated 2x across cluster]
(_ray_fit pid=13941) /usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning:  [repeated 2x across cluster]
(_ray_fit pid=13941) Dask dataframe query planning

(_ray_fit pid=13917) [50]	valid_set's binary_logloss: 0.0724257 [repeated 3x across cluster]


(_ray_fit pid=14014) /usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning:  [repeated 2x across cluster]
(_ray_fit pid=14014) Dask dataframe query planning is disabled because dask-expr is not installed. [repeated 2x across cluster]
(_ray_fit pid=14014)  [repeated 4x across cluster]
(_ray_fit pid=14014) You can install it with `pip install dask[dataframe]` or `conda install dask`. [repeated 2x across cluster]
(_ray_fit pid=14014) This will raise in a future version. [repeated 2x across cluster]
(_ray_fit pid=14014)   warnings.warn(msg, FutureWarning) [repeated 2x across cluster]


(_ray_fit pid=14014) [50]	valid_set's binary_logloss: 0.0657614 [repeated 4x across cluster]


(_dystack pid=13555) Saving /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/LightGBMXT_BAG_L1/utils/oof.pkl
(_dystack pid=13555) Saving /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/LightGBMXT_BAG_L1/model.pkl
(_dystack pid=13555) 	0.8817	 = Validation score   (roc_auc)
(_dystack pid=13555) 	30.74s	 = Training   runtime
(_dystack pid=13555) 	1.08s	 = Validation runtime
(_dystack pid=13555) 	1028.3	 = Inference  throughput (rows/s | 1111 batch size)
(_dystack pid=13555) Saving /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/trainer.pkl
(_dystack pid=13555) Skipping LightGBM_BAG_L1 due to lack of time remaining.
(_dystack pid=13555) Saving /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/trainer.pkl
(_dystack pid=13555) Skipping RandomForestGini_BAG_L1 due to lack of time remaining.
(_dystack pid=13555) Saving /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/trainer.pkl
(_dystack pid=13555) Skipping RandomForestEntr_BAG_L1 due to lack of time remaining.
(_dy

*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val eval_metric  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0         LightGBMXT_BAG_L1   0.890701     roc_auc       0.976684  52.623334                0.976684          52.623334            1      False          1
1       WeightedEnsemble_L3   0.890701     roc_auc       0.979299  52.636115                0.002615           0.012781            3      False          3
2       WeightedEnsemble_L2   0.890701     roc_auc       0.979316  52.637161                0.002633           0.013827            2      False          2
3  WeightedEnsemble_L3_FULL        NaN     roc_auc            NaN   3.215581                     NaN           0.012781            3       True          6
4  WeightedEnsemble_L2_FULL        NaN     roc_auc            NaN   3.216627                     NaN           0.013827            2       True          5
5    Lig

In [7]:
test_identity = pd.read_csv(directory+'test_identity.csv')
test_transaction = pd.read_csv(directory+'test_transaction.csv')
test_data = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')
test_data.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,DeviceType,DeviceInfo
0,3663549,18403224,31.95,W,10409,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3663550,18403263,49.00,W,4272,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3663551,18403310,171.00,W,4476,574.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3663552,18403310,284.95,W,10989,360.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3663553,18403317,67.95,W,18018,452.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
test_data = test_data[:1000] # subset of training dataset to avoid 'Out of memory' error
test_data.shape

(1000, 433)

In [15]:
test_data.rename(columns={f"id-{i:02}": f"id_{i:02}" for i in range(1, 39)}, inplace=True)
test_data.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,3663549,18403224,31.95,W,10409,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3663550,18403263,49.00,W,4272,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3663551,18403310,171.00,W,4476,574.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3663552,18403310,284.95,W,10989,360.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3663553,18403317,67.95,W,18018,452.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
y_pred = predictor.predict(test_data)
y_pred.head()

Loading: /content/AutoGluonModels/models/LightGBMXT_BAG_L1_FULL/model.pkl


,isFraud
0,0
1,0
2,0
3,0
4,0


In [23]:
test_data[label] = y_pred
test_data.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,isFraud
0,3663549.0,18403224.0,31.95,W,10409.0,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,3663550.0,18403263.0,49.00,W,4272.0,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,3663551.0,18403310.0,171.00,W,4476.0,574.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,3663552.0,18403310.0,284.95,W,10989.0,360.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,3663553.0,18403317.0,67.95,W,18018.0,452.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [24]:
predictor.evaluate(test_data)

Loading: /content/AutoGluonModels/models/LightGBMXT_BAG_L1_FULL/model.pkl


{'roc_auc': 0.7956030150753769,
 'accuracy': 0.9975,
 'balanced_accuracy': 0.75,
 'mcc': 0.7062201265667241,
 'f1': 0.6666666666666666,
 'precision': 1.0,
 'recall': 0.5}

In [25]:
predictor.leaderboard(test_data)

Loading: /content/AutoGluonModels/models/LightGBMXT_BAG_L1_FULL/model.pkl
Loading: /content/AutoGluonModels/models/WeightedEnsemble_L2_FULL/model.pkl
Loading: /content/AutoGluonModels/models/WeightedEnsemble_L3_FULL/model.pkl


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT_BAG_L1_FULL,0.795603,NaN,roc_auc,0.043783,NaN,3.202800,0.043783,NaN,3.202800,1,True,4
1,WeightedEnsemble_L3_FULL,0.795603,NaN,roc_auc,0.047775,NaN,3.215581,0.003991,NaN,0.012781,3,True,6
2,WeightedEnsemble_L2_FULL,0.795603,NaN,roc_auc,0.055137,NaN,3.216627,0.011354,NaN,0.013827,2,True,5
3,LightGBMXT_BAG_L1,NaN,0.890701,roc_auc,NaN,0.976684,52.623334,NaN,0.976684,52.623334,1,False,1
4,WeightedEnsemble_L3,NaN,0.890701,roc_auc,NaN,0.979299,52.636115,NaN,0.002615,0.012781,3,False,3
5,WeightedEnsemble_L2,NaN,0.890701,roc_auc,NaN,0.979316,52.637161,NaN,0.002633,0.013827,2,False,2
